In [2]:
import os
os.chdir("/content/drive/MyDrive/ug-project/src")
!pwd
%load_ext autoreload
%autoreload 2

/content/drive/MyDrive/ug-project/src


In [3]:
from datasets import ToyboxDatasetInstances, TOYBOX_MEAN, TOYBOX_STD
import numpy as np
import torchvision.transforms.v2 as v2
import torch.utils.data as torchdata
from torch.utils.data import DataLoader

### Set hyperparameters

In [1]:
learning_rate = (0.005, 0.001)
epochs = 10
batch_size = 256

### Define transforms

In [4]:
rng = np.random.default_rng(seed=5)

prob = 0.2
color_transforms = [v2.RandomApply([v2.ColorJitter(brightness=0.2)], p=prob),
                    v2.RandomApply([v2.ColorJitter(hue=0.2)], p=prob),
                    v2.RandomApply([v2.ColorJitter(saturation=0.2)], p=prob),
                    v2.RandomApply([v2.ColorJitter(contrast=0.2)], p=prob),
                    v2.RandomEqualize(p=prob),
                    v2.RandomPosterize(bits=4, p=prob),
                    v2.RandomAutocontrast(p=prob)
                    ]
transform = v2.Compose([v2.ToPILImage(),
                        v2.Resize((256, 256)),
                        v2.RandomResizedCrop(size=224, scale=(0.5, 1.0), interpolation=v2.InterpolationMode.BICUBIC),
                        v2.RandomOrder(color_transforms),
                        v2.RandomHorizontalFlip(),
                        v2.ToTensor(),
                        v2.Normalize(mean=TOYBOX_MEAN, std=TOYBOX_STD),
                        v2.RandomErasing(p=0.5)
                        ])
transform_test = v2.Compose([
    v2.ToPILImage(),
    v2.Resize(224),
    v2.ToTensor(),
    v2.Normalize(mean=TOYBOX_MEAN, std=TOYBOX_STD)
])

/usr/local/lib/python3.11/dist-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


### Load Toybox dataset

In [5]:
# Checking full dataset size
train_dataset = ToyboxDatasetInstances(rng=rng, train=True, hypertune=True, transform=transform)
print(f"Dev set size: {len(train_dataset)}")

val_dataset = ToyboxDatasetInstances(rng=rng, train=False, hypertune=True, transform=transform_test)
print(f"Val set size: {len(val_dataset)}")

test_dataset = ToyboxDatasetInstances(rng=rng, train=False, hypertune=False, transform=transform_test)
print(f"Test set size: {len(test_dataset)}")

Dev set size: 133644
Val set size: 2820
Test set size: 2820


In [6]:
train_dataset = ToyboxDatasetInstances(rng=rng, train=True, hypertune=True, transform=transform, num_images_per_object=150)
print(f"Dev set size: {len(train_dataset)}")

val_dataset = ToyboxDatasetInstances(rng=rng, train=False, hypertune=True, transform=transform_test)
print(f"Val set size: {len(val_dataset)}")

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

Dev set size: 54000
Val set size: 2820


In [7]:
import torch
import torch.nn as nn
from model import ResNet18Sup

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

criterion = nn.CrossEntropyLoss()
steps = len(train_loader)

Using device: cuda


### Loop through learning rates

In [8]:
for lr in learning_rate:
    print(f"Learning rate: {lr}")
    model = ResNet18Sup(num_classes=360).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)

    warmup_scheduler = torch.optim.lr_scheduler.LinearLR(optimizer=optimizer, start_factor=0.01, end_factor=1.0,
                                                          total_iters=2*steps)
    decay_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max=(epochs - 2) * steps)
    combined_scheduler = torch.optim.lr_scheduler.SequentialLR(optimizer=optimizer,
                                                                schedulers=[warmup_scheduler, decay_scheduler],
                                                                milestones=[2*steps+1])

    # Initialize metrics for ploting
    train_losses, train_correct = [], []
    val_losses, val_correct = [], []
    logs = []

    for epoch in range(0, epochs):

        ###################### Train model #########################
        model.train()
        for idx, images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            logits = model(images)
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()
            combined_scheduler.step()

        # Evaluate on training set
        model.eval()
        train_loss, train_corr, total_train = 0, 0, 0

        with torch.no_grad():
            for _, images, labels in train_loader:
                images, labels = images.to(device), labels.to(device)

                y_pred = model(images)
                loss = criterion(y_pred, labels)

                train_loss += loss.item() * images.size(0)
                predicted = torch.max(y_pred.data, 1)[1]
                train_corr += (predicted == labels).sum().item()
                total_train += labels.size(0)

        avg_train_loss = train_loss / total_train
        train_accuracy = (train_corr / total_train) * 100

        ###################### Evaluate model ######################
        val_loss, val_corr, total_val = 0, 0, 0

        with torch.no_grad():
            for _, images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)

                y_val_pred = model(images)
                loss = criterion(y_val_pred, labels)

                val_loss += loss.item() * images.size(0)
                predicted = torch.max(y_val_pred, 1)[1]
                val_corr += (predicted == labels).sum().item()
                total_val += labels.size(0)


        avg_val_loss = val_loss / total_val
        val_accuracy = (val_corr / total_val) * 100

        # Save results of current epoch
        train_losses.append(avg_train_loss)
        train_correct.append(train_accuracy)
        val_losses.append(avg_val_loss)
        val_correct.append(val_accuracy)

        # Add epoch results to log file
        log_entry = (f"Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.4f}, Train Acc: {train_accuracy:.2f}, Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.2f}%")
        print(log_entry)
        logs.append(log_entry)

Learning rate: 0.005
Epoch 1/10, Train Loss: 3.2747, Train Acc: 29.58, Val Loss: 3.0134, Val Acc: 35.21%
Epoch 2/10, Train Loss: 2.1784, Train Acc: 46.61, Val Loss: 2.1758, Val Acc: 49.08%


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:240: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


Epoch 3/10, Train Loss: 1.2004, Train Acc: 67.68, Val Loss: 1.6760, Val Acc: 61.88%
Epoch 4/10, Train Loss: 0.7573, Train Acc: 79.79, Val Loss: 1.0867, Val Acc: 74.01%
Epoch 5/10, Train Loss: 0.4929, Train Acc: 86.19, Val Loss: 0.9178, Val Acc: 78.76%
Epoch 6/10, Train Loss: 0.3853, Train Acc: 88.95, Val Loss: 0.7890, Val Acc: 81.95%
Epoch 7/10, Train Loss: 0.2007, Train Acc: 94.38, Val Loss: 0.5901, Val Acc: 86.67%
Epoch 8/10, Train Loss: 0.1292, Train Acc: 96.49, Val Loss: 0.4524, Val Acc: 89.36%
Epoch 9/10, Train Loss: 0.0953, Train Acc: 97.41, Val Loss: 0.4460, Val Acc: 89.82%
Epoch 10/10, Train Loss: 0.0882, Train Acc: 97.69, Val Loss: 0.4199, Val Acc: 90.67%
Learning rate: 0.001
Epoch 1/10, Train Loss: 2.8696, Train Acc: 34.55, Val Loss: 2.5398, Val Acc: 40.07%
Epoch 2/10, Train Loss: 1.9406, Train Acc: 51.36, Val Loss: 1.8836, Val Acc: 52.91%
Epoch 3/10, Train Loss: 1.9931, Train Acc: 52.37, Val Loss: 1.9182, Val Acc: 54.96%
Epoch 4/10, Train Loss: 0.9488, Train Acc: 74.74, Val 